## __Speech → Transcript → Call notes__

This Jupyter notebook aims at transforming my meetings' recordings (audio) into calls notes. The steps are: 

1. get the transcript from the audio using [Whisper](https://openai.com/index/whisper/) (via [GroqCloud](https://console.groq.com/docs/speech-to-text)) 

2. create a note call from the transcript using [OpenAI](http://openai.com/)

In [ ]:
import os

os.environ["GROQ_API_KEY"] = "gsk_..."
os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
# SPEECH -> TRANSCRIPTION
from pathlib import Path
from groq import Groq

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
DOWNLOADS = Path.home() / "Downloads"

for audio_path in DOWNLOADS.glob("*.m4a"):
    print(f"\n→ Processing {audio_path.name}...")
    with open(audio_path, "rb") as f:
        data = f.read()         
        transcription = client.audio.transcriptions.create(
            file=(audio_path.name, data),
            model="whisper-large-v3",
            response_format="verbose_json",
        )
    print(f"The Transcript for {audio_path.name} is :\n{transcription.text}")

    out_path = audio_path.with_suffix(".txt")
    out_path.write_text(transcription.text.strip(), encoding="utf-8")
    print(f"Transcript saved to {out_path}")


In [ ]:
# TRANSCRIPTION -> NOTES
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

for transcript_path in DOWNLOADS.glob("*.txt"):
    print(f"\n→ Processing {transcript_path.name}...")
    
    transcript = transcript_path.read_text(encoding="utf-8")

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", 
            "content": (
                "You are an assistant that converts meeting transcripts into concise, "
                "well-structured call notes. Include key points, decisions, and action items."
            )
            },
            {"role": "user",
            "content": f"Here is the transcript of a meeting:\n\n{transcript}\n\n"
                           "Please summarize it into clear call notes."
            }

        ],
        temperature=0.2,
        max_tokens=800,
    )
    
    summary = response.choices[0].message.content.strip()
    out_path = transcript_path.with_suffix(".summary.txt")
    out_path.write_text(summary, encoding="utf-8")

    print(f"Summared notes saved to {out_path.name}")